In [19]:
#load_data
import scipy.io
import numpy as np
import mne
from scipy import signal
data=scipy.io.loadmat('dataset_BCIcomp1.mat')
data_test=data['x_test']
data_train=data['x_train']
label_train=data['y_train'].reshape(1,-1)-1
label=scipy.io.loadmat('y_test.mat')
label_test=label['y_test'].reshape(1,-1)-1
print(label_test.shape)
print(label_train.shape)
y_train=label_train[0]
y_test=label_test[0]
print(y_train.shape)
print(y_test.shape)
b,a=signal.butter(8,[(16/128),(64/128)],'bandpass')
buffer_x_test=signal.filtfilt(b,a,data_test,axis=0)
buffer_x_train=signal.filtfilt(b,a,data_train,axis=0)
print(buffer_x_test.shape)
all_x_train=np.transpose(buffer_x_train,[2,1,0])
all_x_test=np.transpose(buffer_x_test,[2,1,0])
x_train=all_x_train[:,0::2,448:896]
print(x_train.shape)
x_test=all_x_test[:,0::2,448:896]
print(x_test.shape)


(1, 140)
(1, 140)
(140,)
(140,)
(1152, 3, 140)
(140, 2, 448)
(140, 2, 448)


In [21]:
#decompose and reconstruct EEG signals
import pywt
db4=pywt.Wavelet('db4')
cA3,cD3,cD2,cD1= pywt.wavedec(x_train[1,1,:],db4,mode='symmetric',level=3)
print(cD2.shape)
print(cA3.shape)
print(cD1.shape)
print(cD3.shape)
cD2=np.zeros(117)
cA3=np.zeros(62)
cD1=np.zeros(227)
x_zz3=pywt.waverec([cA3,cD3,cD2,cD1],db4)
import pywt
db4=pywt.Wavelet('db4')
def Dwt(X):
    cA3,cD3,cD2,cD1 = pywt.wavedec(X,db4,mode='symmetric',level=3)
    return cA3,cD3,cD2,cD1
def cD3_features(x):
    Bands_D3=np.empty((x.shape[0],x.shape[1],448))
    for i in range(x.shape[0]):
        for ii in range(x.shape[1]):
            cA3,cD3,cD2,cD1=Dwt(x[i,ii,:])
            cA3=np.zeros(62)
            cD2=np.zeros(117)
            cD1=np.zeros(227)
            Bands_D3[i,ii,:]=pywt.waverec([cA3,cD3,cD2,cD1],db4)
    return Bands_D3
def cD2_features(x):
    Bands_D2=np.empty((x.shape[0],x.shape[1],448))
    for i in range(x.shape[0]):
        for ii in range(x.shape[1]):
            cA3,cD3,cD2,cD1=Dwt(x[i,ii,:])
            cA3=np.zeros(62)
            cD3=np.zeros(62)
            cD1=np.zeros(227)
            Bands_D2[i,ii,:]=pywt.waverec([cA3,cD3,cD2,cD1],db4)
    return Bands_D2
x_train_d3=cD3_features(x_train)
x_train_d2=cD2_features(x_train)
x_test_d3=cD3_features(x_test)
x_test_d2=cD2_features(x_test)
print(x_train_d3.shape)
print(x_test_d3.shape)
print(x_train_d2.shape)
print(x_test_d2.shape)

(117,)
(62,)
(227,)
(62,)
(140, 2, 448)
(140, 2, 448)
(140, 2, 448)
(140, 2, 448)


In [23]:
!pip install statsmodels

In [25]:
#calcutue AR coef of each  channel in EEG signals of different frequency bands
from statsmodels.tsa.ar_model import AR
def get_ARcoef(x):
    model=AR(x)
    model_fit=model.fit(maxlag=5)
    coef=model_fit.params
    return coef
def get_features(x,y):
    for i in range(140):
        for j in range(2):
            y[i,j]=get_ARcoef(x[i,j])
    return y

In [38]:
import pandas as pd
d3_train_features=pd.DataFrame(X_train)
d3_train_features.to_csv('features.csv')
d3_train_coef=np.zeros((140,2,6))
d3_test_coef=np.zeros((140,2,6))
d2_train_coef=np.zeros((140,2,6))
d2_test_coef=np.zeros((140,2,6))
d3_train_features=get_features(x_train_d3,d3_train_coef)
d2_train_features=get_features(x_train_d2,d2_train_coef)
d3_test_features=get_features(x_test_d3,d3_test_coef)
d2_test_features=get_features(x_test_d2,d2_test_coef)
print(d3_train_features.shape)
print(d3_train_features)


c:\users\medha\appdata\local\programs\python\python38-32\lib\site-packages\statsmodels\tsa\ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPR

(140, 2, 6)
[[[ 1.96683954e-05  1.58759914e+00 -1.11957103e+00  4.59665393e-01
   -3.82018887e-01  1.17173806e-01]
  [ 4.08353474e-05  1.48225280e+00 -9.03157835e-01  2.83093576e-01
   -4.18375501e-01  2.50162312e-01]]

 [[-7.98295629e-05  1.63455534e+00 -1.18856285e+00  5.18766688e-01
   -3.84802769e-01  9.91864183e-02]
  [ 1.80103329e-05  1.69647431e+00 -1.25681767e+00  5.66221989e-01
   -3.78360584e-01  8.72372463e-02]]

 [[-6.76698512e-05  1.61662872e+00 -1.15397453e+00  4.83491408e-01
   -3.79288143e-01  1.11062174e-01]
  [-5.67337718e-05  1.60407548e+00 -1.06896089e+00  3.77826494e-01
   -3.64049489e-01  1.73162168e-01]]

 ...

 [[-4.61211296e-05  1.64852091e+00 -1.18256573e+00  4.95813617e-01
   -3.72096087e-01  1.10573550e-01]
  [-4.93583614e-05  1.70519528e+00 -1.30129908e+00  6.31858330e-01
   -3.99846770e-01  6.71370801e-02]]

 [[ 2.45310797e-05  1.68419427e+00 -1.25819491e+00  5.81020249e-01
   -3.87111966e-01  7.98037081e-02]
  [ 1.95238822e-05  1.71360453e+00 -1.30543531e

In [39]:
#concatenate and normalize the coef as extracted features
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
acc=[]
ss = preprocessing.StandardScaler()
X_train=ss.fit_transform(np.concatenate((d3_train_features[:,0,:],d3_train_features[:,1,:],d2_train_features[:,0,:],d2_train_features[:,1,:]),axis=1))
X_test=ss.transform(np.concatenate((d3_test_features[:,0,:],d3_test_features[:,1,:],d2_test_features[:,0,:],d2_test_features[:,1,:]),axis=1))
print(X_train.shape)
print(X_test.shape)
print(X_train)
print(X_test)

(140, 24)
(140, 24)
[[ 0.36055183 -0.520549    0.28248871 ...  0.90562401 -0.05478024
  -0.29748046]
 [-1.35625364  0.2382766  -0.27809691 ...  1.01998547 -1.8060067
   1.6353143 ]
 [-1.1464417  -0.05142263  0.00294678 ...  0.19150651  0.12655755
  -0.36901726]
 ...
 [-0.7746254   0.46396425 -0.22936797 ...  0.87649892 -1.17139276
   0.97761816]
 [ 0.4444559   1.04045591 -0.84388477 ...  0.87443441  0.17844861
  -0.80602261]
 [-1.03773912  0.96262712 -1.00213177 ... -1.43234994  0.45259185
  -0.17353459]]
[[ 0.27193525 -3.20229248  2.89871703 ... -0.4130415   1.04733446
  -1.05252966]
 [-0.51568197 -1.08754206  1.22688352 ...  0.31634495 -0.07345964
  -0.21373256]
 [-1.06452569  0.66150511 -0.49528924 ...  0.8019574  -0.88537507
   0.8575252 ]
 ...
 [-1.19796409  0.75175089 -0.76834107 ... -0.34577342  0.40495993
  -0.41171417]
 [ 0.47244712  0.57474256 -0.80423205 ... -0.02603259  0.22917936
  -0.1977518 ]
 [-0.12774406  0.49144038 -0.63167836 ...  0.45496229 -0.56287791
   0.63094661